In [77]:
## Part-Of-Speech Tagging 
# http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.102.4921&rep=rep1&type=pdf

# Using Stanford NLP Stanza and Jieba
# https://github.com/stanfordnlp/stanza/
# https://stanfordnlp.github.io/stanza/data_objects.html#word

import bucc_proc as bp

np = bp.np # Numpy
pd = bp.pd # Pandas
Path = bp.Path # Pathlib
from random import randrange

import importlib
importlib.reload(bp)

import jieba
import stanza

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
stanza.download('en')
stanza.download('zh')

2020-06-18 11:18:48 INFO: Downloading default packages for language: en (English)...
2020-06-18 11:18:52 INFO: File exists: C:\Users\gabri\stanza_resources\en\default.zip.
2020-06-18 11:19:33 INFO: Finished downloading models and saved to C:\Users\gabri\stanza_resources.
2020-06-18 11:19:34 INFO: "zh" is an alias for "zh-hans"
2020-06-18 11:19:34 INFO: Downloading default packages for language: zh-hans (Simplified_Chinese)...
2020-06-18 11:20:02 INFO: File exists: C:\Users\gabri\stanza_resources\zh-hans\default.zip.
2020-06-18 11:20:51 INFO: Finished downloading models and saved to C:\Users\gabri\stanza_resources.


In [3]:
en_postagger = stanza.Pipeline('en')

2020-06-18 11:20:51 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
| ner       | ontonotes |

2020-06-18 11:20:52 INFO: Use device: cpu
2020-06-18 11:20:52 INFO: Loading: tokenize
2020-06-18 11:20:53 INFO: Loading: pos
2020-06-18 11:21:00 INFO: Loading: lemma
2020-06-18 11:21:01 INFO: Loading: depparse
2020-06-18 11:21:04 INFO: Loading: ner
2020-06-18 11:21:15 INFO: Done loading processors!


In [4]:
zh_postagger = stanza.Pipeline('zh')

2020-06-18 11:21:15 INFO: "zh" is an alias for "zh-hans"
2020-06-18 11:21:15 INFO: Loading these models for language: zh-hans (Simplified_Chinese):
| Processor | Package   |
-------------------------
| tokenize  | gsdsimp   |
| pos       | gsdsimp   |
| lemma     | gsdsimp   |
| depparse  | gsdsimp   |
| ner       | ontonotes |

2020-06-18 11:21:15 INFO: Use device: cpu
2020-06-18 11:21:15 INFO: Loading: tokenize
2020-06-18 11:21:16 INFO: Loading: pos
2020-06-18 11:21:27 INFO: Loading: lemma
2020-06-18 11:21:27 INFO: Loading: depparse
2020-06-18 11:21:40 INFO: Loading: ner
2020-06-18 11:21:48 INFO: Done loading processors!


In [38]:
zh_postagger_pretoken = stanza.Pipeline(lang='zh', tokenize_pretokenized=True)

# words split by space, sentences by newline

2020-06-18 13:25:45 INFO: "zh" is an alias for "zh-hans"
2020-06-18 13:25:45 INFO: Loading these models for language: zh-hans (Simplified_Chinese):
| Processor | Package   |
-------------------------
| tokenize  | gsdsimp   |
| pos       | gsdsimp   |
| lemma     | gsdsimp   |
| depparse  | gsdsimp   |
| ner       | ontonotes |

2020-06-18 13:25:46 INFO: Use device: cpu
2020-06-18 13:25:46 INFO: Loading: tokenize
2020-06-18 13:25:46 INFO: Loading: pos
2020-06-18 13:26:14 INFO: Loading: lemma
2020-06-18 13:26:16 INFO: Loading: depparse
2020-06-18 13:26:58 INFO: Loading: ner
2020-06-18 13:27:14 INFO: Done loading processors!


In [5]:
new_df = bp.get_merge()
new_df.count

Merged file exists, reading...


<bound method DataFrame.count of              ID_zh         ID_en                             Sentence_zh  \
0     zh-000000033  en-000005983               1989年以前，全球经济包含大约8亿到10亿人口。   
1     zh-000000231  en-000047360        今日全球面临的威胁是超民族的，因此也必须采取超民族的方式来应对。   
2     zh-000000272  en-000027140                   欧盟移民政策的硬伤还有一个不太显著的方面。   
3     zh-000000438  en-000065621           只有让民粹主义服务于自由主义改革，政府才能取得长久的利益。   
4     zh-000000639  en-000005169       但社会民主派必须理解为何示威的发展会独立于现有的有组织中左翼政治。   
...            ...           ...                                     ...   
1848  zh-000094590  en-000013258         事件发生后当局在尚未进行调查的情况下就匆匆掩埋了出事列车残骸。   
1849  zh-000094593  en-000061419             北方拥有丰富的自然资源，就连电力也是从北方输送到南方。   
1850  zh-000094607  en-000039373                如果利率为3%，那么年税收额必须增加15亿美元。   
1851  zh-000094611  en-000003807           五年前，叙利亚北部边陲城镇享受着土耳其高速经济增长的红利。   
1852  zh-000094633  en-000083972  在过去的一个世纪中，我们的世界发生了翻天覆地的变化——技术是其中的重要原因。   

                                            Sentence_e

In [49]:
def make_pos_counts(sentence, attr = "upos", lang ="en", filtered=[]):
    
    tag_count_dict = {}
    
    if lang == "en":
        pos_list = [getattr(word, attr) for s in en_postagger(sentence).sentences for word in s.words]
        
    elif lang == "zh":
        pos_list = [getattr(word, attr) for s in zh_postagger(sentence).sentences for word in s.words]
        
    elif lang == "zh_jieba":
        pos_list = [getattr(word, attr) for s in zh_postagger(' '.join(jieba.lcut(sentence))).sentences for word in s.words]
        
    for tag in pos_list:
        if tag not in tag_count_dict:
            tag_count_dict[tag]=1
        else: 
            tag_count_dict[tag]+=1
    
    if filtered:
        filtered_dict = {k: v for k, v in tag_count_dict.items() if k in filtered}
        output_list = sorted(filtered_dict.items())
    else:
        output_list = sorted(tag_count_dict.items())
    
    
    return output_list

see_tags = ['NOUN','VERB','ADJ','NUM','PROPN']
make_pos_counts('I’m sure that this world – I think I’ll have fully enjoyed it', filtered=see_tags)

[('ADJ', 1), ('NOUN', 1), ('VERB', 2)]

In [46]:
filters=[]

pos_df = new_df
pos_df['pos_en'] = [make_pos_counts(pos_df.iloc[i]['Sentence_en'], "upos", "en", filtered=filters) for i in range(1853)]
pos_df['pos_zh'] = [make_pos_counts(pos_df.iloc[i]['Sentence_zh'], "upos", "zh", filtered=filters) for i in range(1853)]

In [47]:
pos_df

,ID_zh,ID_en,Sentence_zh,Sentence_en,pos_en,pos_zh
0,zh-000000033,en-000005983,1989年以前，全球经济包含大约8亿到10亿人口。,"Until 1989, the global market encompassed betw...","[(ADJ, 1), (ADP, 2), (CCONJ, 1), (DET, 1), (NO...","[(ADP, 1), (ADV, 1), (NOUN, 4), (NUM, 3), (PAR..."
1,zh-000000231,en-000047360,今日全球面临的威胁是超民族的，因此也必须采取超民族的方式来应对。,The threats facing the world today are suprana...,"[(ADJ, 2), (ADV, 2), (AUX, 3), (DET, 2), (NOUN...","[(ADP, 1), (ADV, 2), (AUX, 2), (NOUN, 6), (PAR..."
2,zh-000000272,en-000027140,欧盟移民政策的硬伤还有一个不太显著的方面。,"There is another, less obvious, reason why the...","[(ADJ, 2), (ADP, 1), (ADV, 2), (AUX, 1), (DET,...","[(ADJ, 1), (ADV, 3), (NOUN, 5), (NUM, 1), (PAR..."
3,zh-000000438,en-000065621,只有让民粹主义服务于自由主义改革，政府才能取得长久的利益。,Only if populism is put at the service of libe...,"[(ADJ, 2), (ADP, 2), (ADV, 1), (AUX, 2), (DET,...","[(ADJ, 1), (ADP, 1), (AUX, 1), (NOUN, 7), (PAR..."
4,zh-000000639,en-000005169,但社会民主派必须理解为何示威的发展会独立于现有的有组织中左翼政治。,But social democrats must understand why the p...,"[(ADJ, 2), (ADP, 1), (ADV, 3), (AUX, 1), (CCON...","[(ADP, 1), (ADV, 1), (AUX, 2), (NOUN, 6), (PAR..."
...,...,...,...,...,...,...
1848,zh-000094590,en-000013258,事件发生后当局在尚未进行调查的情况下就匆匆掩埋了出事列车残骸。,The wrecked body of the ruined train was burie...,"[(ADP, 2), (ADV, 2), (AUX, 1), (DET, 3), (NOUN...","[(ADP, 3), (ADV, 4), (NOUN, 6), (PART, 2), (PU..."
1849,zh-000094593,en-000061419,北方拥有丰富的自然资源，就连电力也是从北方输送到南方。,"Natural resources were abundant in the North, ...","[(ADJ, 2), (ADP, 3), (ADV, 1), (AUX, 2), (CCON...","[(ADJ, 1), (ADP, 1), (ADV, 2), (NOUN, 6), (PAR..."
1850,zh-000094607,en-000039373,如果利率为3%，那么年税收额必须增加15亿美元。,"If it is 3%, the required increase in annual t...","[(ADJ, 1), (ADP, 2), (AUX, 1), (DET, 2), (NOUN...","[(ADP, 1), (ADV, 1), (AUX, 2), (NOUN, 4), (NUM..."
1851,zh-000094611,en-000003807,五年前，叙利亚北部边陲城镇享受着土耳其高速经济增长的红利。,"Five years ago, Syria’s northern border towns ...","[(ADJ, 2), (ADP, 1), (ADV, 1), (AUX, 1), (DET,...","[(ADJ, 1), (ADP, 1), (NOUN, 6), (NUM, 1), (PAR..."


In [45]:
i = 0
filters = ['NOUN', 'VERB', 'ADJ', 'NUM']

doc= en_postagger(pos_df.iloc[i]['Sentence_en'])
print(*[f'word: {word.text}\tupos: {word.upos}\txpos: {word.xpos}\tfeats: {word.feats if word.feats else "_"}' for sent in doc.sentences for word in sent.words], sep='\n')
print(make_pos_counts(pos_df.iloc[i]['Sentence_en'], "upos", "en", filtered=filters))

print()

doc= zh_postagger(pos_df.iloc[i]['Sentence_zh'])
print(*[f'word: {word.text}\tupos: {word.upos}\txpos: {word.xpos}\tfeats: {word.feats if word.feats else "_"}' for sent in doc.sentences for word in sent.words], sep='\n')
print(make_pos_counts(pos_df.iloc[i]['Sentence_zh'], "upos", "zh", filtered=filters))

print()

doc= zh_postagger_pretoken(' '.join(jieba.lcut(pos_df.iloc[i]['Sentence_zh'])))
print(*[f'word: {word.text}\tupos: {word.upos}\txpos: {word.xpos}\tfeats: {word.feats if word.feats else "_"}' for sent in doc.sentences for word in sent.words], sep='\n')
print(make_pos_counts(pos_df.iloc[i]['Sentence_zh'], "upos", "zh_jieba", filtered=filters))

word: Until	upos: ADP	xpos: IN	feats: _
word: 1989	upos: NUM	xpos: CD	feats: NumType=Card
word: ,	upos: PUNCT	xpos: ,	feats: _
word: the	upos: DET	xpos: DT	feats: Definite=Def|PronType=Art
word: global	upos: ADJ	xpos: JJ	feats: Degree=Pos
word: market	upos: NOUN	xpos: NN	feats: Number=Sing
word: encompassed	upos: VERB	xpos: VBD	feats: Mood=Ind|Tense=Past|VerbForm=Fin
word: between	upos: ADP	xpos: IN	feats: _
word: 800	upos: NUM	xpos: CD	feats: NumType=Card
word: million	upos: NUM	xpos: CD	feats: NumType=Card
word: and	upos: CCONJ	xpos: CC	feats: _
word: one	upos: NUM	xpos: CD	feats: NumType=Card
word: billion	upos: NUM	xpos: CD	feats: NumType=Card
word: people	upos: NOUN	xpos: NNS	feats: Number=Plur
word: .	upos: PUNCT	xpos: .	feats: _
[('ADJ', 1), ('NOUN', 2), ('NUM', 5), ('VERB', 1)]

word: 1989	upos: NUM	xpos: CD	feats: NumType=Card
word: 年	upos: NOUN	xpos: NNB	feats: _
word: 以前	upos: ADP	xpos: IN	feats: _
word: ，	upos: PUNCT	xpos: ,	feats: _
word: 全球	upos: NOUN	xpos: NN	feats: _
wo

In [95]:
def make_pos_vect(pos_list, filters=['NOUN', 'VERB', 'ADJ', 'NUM']):
    
    filtered_dict = {}
    ordered_list = []
    
    for t, n in pos_list:
        if t in filters:
            filtered_dict[t] = n
            
    for f in filters:
        if f not in filtered_dict:
            filtered_dict[f] = 0
            
    ordered_list = [v for k, v in sorted(filtered_dict.items())]
            
    vector = np.asarray(ordered_list)
    
    return vector

In [84]:
def cosine_similarity(v1,v2):
    '''cosine_similarity(transformed_docs[2], transformed_docs[2])'''
    ## Idk why need to np.squeeze (1,148) into (148,) shape to dot product [error: shapes not aligned]
    ## toarray() [error: dimension mismatch] v1.toarray()v2.toarray()
    v1 = np.squeeze(v1)
    v2 = np.squeeze(v2)
    return np.dot(v1,v2) / ( np.sqrt(np.dot(v1,v1)) * np.sqrt(np.dot(v2,v2)) )

In [76]:
filters = ['NOUN', 'VERB', 'ADJ', 'NUM']


pos_df['similarity'] = [cosine_similarity( make_pos_vect(pos_df.iloc[i]['pos_en'], filters), make_pos_vect(pos_df.iloc[i]['pos_zh'], filters) ) for i in range(1853)]

C:\Users\gabri\anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


In [86]:
pos_df

,ID_zh,ID_en,Sentence_zh,Sentence_en,pos_en,pos_zh,similarity
0,zh-000000033,en-000005983,1989年以前，全球经济包含大约8亿到10亿人口。,"Until 1989, the global market encompassed betw...","[(ADJ, 1), (ADP, 2), (CCONJ, 1), (DET, 1), (NO...","[(ADP, 1), (ADV, 1), (NOUN, 4), (NUM, 3), (PAR...",0.845364
1,zh-000000231,en-000047360,今日全球面临的威胁是超民族的，因此也必须采取超民族的方式来应对。,The threats facing the world today are suprana...,"[(ADJ, 2), (ADV, 2), (AUX, 3), (DET, 2), (NOUN...","[(ADP, 1), (ADV, 2), (AUX, 2), (NOUN, 6), (PAR...",0.867722
2,zh-000000272,en-000027140,欧盟移民政策的硬伤还有一个不太显著的方面。,"There is another, less obvious, reason why the...","[(ADJ, 2), (ADP, 1), (ADV, 2), (AUX, 1), (DET,...","[(ADJ, 1), (ADV, 3), (NOUN, 5), (NUM, 1), (PAR...",0.909137
3,zh-000000438,en-000065621,只有让民粹主义服务于自由主义改革，政府才能取得长久的利益。,Only if populism is put at the service of libe...,"[(ADJ, 2), (ADP, 2), (ADV, 1), (AUX, 2), (DET,...","[(ADJ, 1), (ADP, 1), (AUX, 1), (NOUN, 7), (PAR...",0.964237
4,zh-000000639,en-000005169,但社会民主派必须理解为何示威的发展会独立于现有的有组织中左翼政治。,But social democrats must understand why the p...,"[(ADJ, 2), (ADP, 1), (ADV, 3), (AUX, 1), (CCON...","[(ADP, 1), (ADV, 1), (AUX, 2), (NOUN, 6), (PAR...",0.942809
...,...,...,...,...,...,...,...
1848,zh-000094590,en-000013258,事件发生后当局在尚未进行调查的情况下就匆匆掩埋了出事列车残骸。,The wrecked body of the ruined train was burie...,"[(ADP, 2), (ADV, 2), (AUX, 1), (DET, 3), (NOUN...","[(ADP, 3), (ADV, 4), (NOUN, 6), (PART, 2), (PU...",0.995893
1849,zh-000094593,en-000061419,北方拥有丰富的自然资源，就连电力也是从北方输送到南方。,"Natural resources were abundant in the North, ...","[(ADJ, 2), (ADP, 3), (ADV, 1), (AUX, 2), (CCON...","[(ADJ, 1), (ADP, 1), (ADV, 2), (NOUN, 6), (PAR...",0.824163
1850,zh-000094607,en-000039373,如果利率为3%，那么年税收额必须增加15亿美元。,"If it is 3%, the required increase in annual t...","[(ADJ, 1), (ADP, 2), (AUX, 1), (DET, 2), (NOUN...","[(ADP, 1), (ADV, 1), (AUX, 2), (NOUN, 4), (NUM...",0.956183
1851,zh-000094611,en-000003807,五年前，叙利亚北部边陲城镇享受着土耳其高速经济增长的红利。,"Five years ago, Syria’s northern border towns ...","[(ADJ, 2), (ADP, 1), (ADV, 1), (AUX, 1), (DET,...","[(ADJ, 1), (ADP, 1), (NOUN, 6), (NUM, 1), (PAR...",0.976190


In [85]:
filters = ['NOUN', 'VERB', 'ADJ', 'NUM']

for i in range(1853):
    print(i, '\t', cosine_similarity(make_pos_vect(pos_df.iloc[i]['pos_en'], filters),make_pos_vect(pos_df.iloc[i]['pos_zh'], filters)))

0 	 0.8453639444116164
1 	 0.8677218312746245
2 	 0.9091372900969896
3 	 0.9642365197998375
4 	 0.9428090415820634
5 	 0.808378958421306
6 	 0.9741763740941047
7 	 0.9789813881808778
8 	 0.9213068794097575
9 	 0.7730206825239258
10 	 0.8968700041677191
11 	 0.7909115788387002
12 	 0.9355285539274978
13 	 0.46188021535170054
14 	 0.839832050383206
15 	 0.9456912729880023
16 	 0.9491095062382652
17 	 0.9906801321840805
18 	 0.8941795301365222
19 	 0.8869685939836239
20 	 0.9091372900969897
21 	 0.8861418804606043
22 	 0.6041585400191987
23 	 0.8539864924534399
24 	 0.9125550022133407
25 	 0.49613893835683387
26 	 0.9393364366277241
27 	 0.9335200560186733
28 	 0.9559730182892108
29 	 0.8581163303210331
30 	 0.8907235428302466
31 	 0.9518172686249524
32 	 0.9843241382880895
33 	 0.8528028654224419
34 	 0.9860251439617607
35 	 0.7652164912392712
36 	 0.9260923597695477
37 	 0.9799578870122229
38 	 0.23923566684867
39 	 0.8580846402802882
40 	 0.858395075278952
41 	 0.9296696802013682
42 	 

C:\Users\gabri\anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys



426 	 0.8333333333333335
427 	 0.9467621366623017
428 	 0.8717797887081347
429 	 0.7692191581126585
430 	 0.8700221858486124
431 	 0.9003661402650394
432 	 0.8914849883204187
433 	 0.7881264972582642
434 	 0.9258200997725513
435 	 0.9281909617845143
436 	 0.8468303183574205
437 	 0.8834522085987724
438 	 0.7868894753646337
439 	 0.7845390492237075
440 	 0.9529827388934864
441 	 0.9463495102642552
442 	 0.8558411920581226
443 	 0.8082903768654761
444 	 0.824621125123532
445 	 0.6410159650938821
446 	 0.6997640269267261
447 	 0.9351125124928961
448 	 0.8981462390204986
449 	 0.9128709291752769
450 	 0.9438931043083171
451 	 0.6905413026317789
452 	 0.9241377067782293
453 	 0.9255480797579322
454 	 0.8944271909999159
455 	 0.9329573177402843
456 	 0.9405816697932298
457 	 0.9297198959726097
458 	 0.9179360088212722
459 	 0.727392967453308
460 	 0.8981462390204986
461 	 0.8240419241993677
462 	 0.9772545497599154
463 	 0.8804087784801331
464 	 0.8628387846605373
465 	 0.9101820546182064
4

1086 	 0.8952737857857513
1087 	 0.6868295951599089
1088 	 0.7874992309581578
1089 	 0.9941348467724341
1090 	 0.7071067811865475
1091 	 0.6482037235521644
1092 	 0.8972006153026083
1093 	 0.9415130835240085
1094 	 0.7735737130957593
1095 	 0.9128709291752769
1096 	 0.9666666666666667
1097 	 0.9882384840881372
1098 	 0.5692099788303082
1099 	 0.9701425001453319
1100 	 0.8744746321952063
1101 	 0.9706476511819816
1102 	 0.9513029883089883
1103 	 0.9737289911202953
1104 	 0.9797958971132713
1105 	 0.8084520834544433
1106 	 0.8432740427115678
1107 	 0.9697622757528539
1108 	 0.9828721869343219
1109 	 0.9636241116594316
1110 	 0.8731282501307988
1111 	 0.8549867431734429
1112 	 0.9401267913629436
1113 	 0.9601136296387952
1114 	 0.9153857298880941
1115 	 0.7648808633001188
1116 	 0.9544271444636666
1117 	 0.9007546982220901
1118 	 0.9979402655783318
1119 	 0.8894991799933215
1120 	 0.5006261743217588
1121 	 0.8498365855987975
1122 	 0.7907050070875207
1123 	 0.8155908717405552
1124 	 0.8
1

1704 	 0.9052038109696288
1705 	 0.8942581600637
1706 	 0.8957179548750233
1707 	 0.810679228399881
1708 	 0.9486832980505138
1709 	 0.9091372900969896
1710 	 0.8997354108424374
1711 	 0.9124255176195217
1712 	 0.8731282501307986
1713 	 0.9245003270420487
1714 	 0.9622504486493763
1715 	 0.9570377672873267
1716 	 0.9620942898608359
1717 	 0.9116846116771036
1718 	 0.5475708968190233
1719 	 0.965594379407439
1720 	 0.6565321642986127
1721 	 0.9875414397573881
1722 	 0.9169680745952878
1723 	 0.916261362123783
1724 	 0.9074425108007848
1725 	 0.9847319278346618
1726 	 0.7321141582036941
1727 	 0.973328526784575
1728 	 0.9299811099505542
1729 	 0.8155908717405552
1730 	 0.7304267703586719
1731 	 0.9548637106322311
1732 	 0.8769028133059115
1733 	 0.9333333333333333
1734 	 0.9767576910715874
1735 	 0.9341987329938275
1736 	 0.9417419115948374
1737 	 0.8444006618414982
1738 	 0.827170191868511
1739 	 0.8970852271450603
1740 	 0.8908708063747479
1741 	 0.7932043159238391
1742 	 0.93242370648

In [80]:
t = 50
n = 0

while t > n:
    x = randrange(1853)
    y = randrange(1853)
    if x != y:
        z = cosine_similarity(make_pos_vect(pos_df.iloc[x]['pos_en'], filters),make_pos_vect(pos_df.iloc[y]['pos_zh'], filters))
        zz = 'YES' if z>0.9 else 'NO'
    
        print('Line EN:',x,'\tLine ZH:',y,'\tSim:',round(z,2), '\tPair:', zz)
    n+=1

Line EN: 1462 	Line ZH: 672 	Sim: 0.81 	Pair: NO
Line EN: 630 	Line ZH: 235 	Sim: 0.69 	Pair: NO
Line EN: 1072 	Line ZH: 653 	Sim: 0.73 	Pair: NO
Line EN: 1738 	Line ZH: 1030 	Sim: 0.79 	Pair: NO
Line EN: 1662 	Line ZH: 403 	Sim: 0.96 	Pair: YES
Line EN: 1270 	Line ZH: 969 	Sim: 0.38 	Pair: NO
Line EN: 965 	Line ZH: 1017 	Sim: 0.92 	Pair: YES
Line EN: 1455 	Line ZH: 269 	Sim: 0.8 	Pair: NO
Line EN: 764 	Line ZH: 1095 	Sim: 0.94 	Pair: YES
Line EN: 938 	Line ZH: 1119 	Sim: 0.87 	Pair: NO
Line EN: 1066 	Line ZH: 725 	Sim: 0.95 	Pair: YES
Line EN: 221 	Line ZH: 1188 	Sim: 0.81 	Pair: NO
Line EN: 1783 	Line ZH: 581 	Sim: 0.69 	Pair: NO
Line EN: 1087 	Line ZH: 66 	Sim: 0.98 	Pair: YES
Line EN: 363 	Line ZH: 1039 	Sim: 0.72 	Pair: NO
Line EN: 39 	Line ZH: 935 	Sim: 0.85 	Pair: NO
Line EN: 266 	Line ZH: 1714 	Sim: 0.89 	Pair: NO
Line EN: 299 	Line ZH: 1684 	Sim: 0.83 	Pair: NO
Line EN: 1389 	Line ZH: 1772 	Sim: 0.83 	Pair: NO
Line EN: 75 	Line ZH: 777 	Sim: 0.97 	Pair: YES
Line EN: 1214 	Line

In [87]:


def test(n, x=0, proc=True):
    string = ''
    
    tp=0 #true positives
    fn=0 #false negatives
    
    for i in range(x, n):
        this_value = pos_df.iloc[i]['similarity']
        
        if i == x:
            most = this_value
            least = this_value
            most_i = x
            least_i = x
            
        elif most <= this_value:
            most = this_value
            most_i = i
            
        elif least >= this_value:
            least = this_value
            least_i = i
        
        if this_value>0.7:
            j = 'YES'
            tp+=1
        else:
            j = 'NO'
            fn+=1
        
        string = string + str(i) +  "\t{:.2f}".format(round(this_value,2)) + '\t' + j + '\n'
    
    
    string = 'Line\tSimilarity\tPair\n' + string
    string = 'Most:' + str(most_i) + "\t{:.2f}".format(most) + '\n' + string
    string = 'Least:' + str(least_i) + "\t{:.2f}".format(least) + '\n' + string
    string = 'Size:' + str(n-x) + '\tTP:' + str(tp) + '\tFN:'+ str(fn) + '\tAccuracy:' + str(round(tp/(n-x)*100,1))+'%\n' + string
    return string
            
def str_to_txt(filename, string):
    with open(filename,'w', encoding='utf-8') as file:
        file.write(string)
                
#str_to_txt('laser_output.txt',test(1853))
#str_to_txt('laser_output_np.txt',test(1853, proc=False))

print(test(1853))

Size:1853	TP:1737	FN:116	Accuracy:93.7%
Least:1264	0.16
Most:1360	1.00
Line	Similarity	Pair
0	0.85	YES
1	0.87	YES
2	0.91	YES
3	0.96	YES
4	0.94	YES
5	0.81	YES
6	0.97	YES
7	0.98	YES
8	0.92	YES
9	0.77	YES
10	0.90	YES
11	0.79	YES
12	0.94	YES
13	0.46	NO
14	0.84	YES
15	0.95	YES
16	0.95	YES
17	0.99	YES
18	0.89	YES
19	0.89	YES
20	0.91	YES
21	0.89	YES
22	0.60	NO
23	0.85	YES
24	0.91	YES
25	0.50	NO
26	0.94	YES
27	0.93	YES
28	0.96	YES
29	0.86	YES
30	0.89	YES
31	0.95	YES
32	0.98	YES
33	0.85	YES
34	0.99	YES
35	0.77	YES
36	0.93	YES
37	0.98	YES
38	0.24	NO
39	0.86	YES
40	0.86	YES
41	0.93	YES
42	0.98	YES
43	0.88	YES
44	0.92	YES
45	0.84	YES
46	1.00	YES
47	0.61	NO
48	0.75	YES
49	0.98	YES
50	0.89	YES
51	0.96	YES
52	0.75	YES
53	0.93	YES
54	0.97	YES
55	1.00	YES
56	0.93	YES
57	0.70	YES
58	0.92	YES
59	0.92	YES
60	0.93	YES
61	0.75	YES
62	0.98	YES
63	0.85	YES
64	0.91	YES
65	0.79	YES
66	0.94	YES
67	0.98	YES
68	0.71	YES
69	0.88	YES
70	1.00	YES
71	0.86	YES
72	0.59	NO
73	0.60	NO
74	0.97	YES
75	0.99	YES
76	0.88	YES
7

In [106]:
c_df = pd.DataFrame()
c_df['Line_en'] = pos_df.index
c_df['Line_zh'] = pos_df.index
c_df['similarity'] = pos_df['similarity']
c_df

,Line_en,Line_zh,similarity
0,0,0,0.845364
1,1,1,0.867722
2,2,2,0.909137
3,3,3,0.964237
4,4,4,0.942809
...,...,...,...
1848,1848,1848,0.995893
1849,1849,1849,0.824163
1850,1850,1850,0.956183
1851,1851,1851,0.976190


In [105]:
inc_file = Path("zh-en.training.incorrect")
if inc_file.is_file():
    print('INC File exists, reading...')
    inc_df = pd.read_csv(inc_file, header=0, sep='\t')

else:
    print('No inc file Error')
    

if 'similarity' not in inc_df.columns:
    cs_list = []

    for i in range(1853):
        x = int(inc_df.iloc[i]['Line_en'])
        y = int(inc_df.iloc[i]['Line_zh'])
        cs_list.append(cosine_similarity(make_pos_vect(pos_df.iloc[x]['pos_en']), make_pos_vect(pos_df.iloc[y]['pos_zh'])))

    inc_df['similarity'] = cs_list

    inc_df.to_csv(inc_file, index=False, sep='\t')

inc_df

INC File exists, reading...


,Line_en,Line_zh,length_en,length_zh,similarity
0,647,430,126,52,0.800499
1,1835,1362,56,69,0.737865
2,365,15,80,64,0.421160
3,1373,701,89,43,0.966667
4,197,987,114,60,0.862924
...,...,...,...,...,...
1848,1308,962,93,64,0.736248
1849,801,602,108,34,0.908893
1850,1778,1359,91,58,0.980379
1851,1782,1097,78,72,0.888532


In [115]:
def _f_score(x, cut_point):
    p,n = (0,0)
    
    if x['similarity'] > cut_point:
        p += 1
    else:
        n +=1
    return p, n

def f_score(cut_point):
    
    tp = c_df.apply(lambda x: _f_score(x, cut_point), axis=1).value_counts().get((1,0))
    fn = c_df.apply(lambda x: _f_score(x, cut_point), axis=1).value_counts().get((0,1))
    fp = inc_df.apply(lambda x: _f_score(x, cut_point), axis=1).value_counts().get((1,0))
    tn = inc_df.apply(lambda x: _f_score(x, cut_point), axis=1).value_counts().get((0,1))
    
    tp = tp if tp else 0
    fn = fn if fn else 0
    fp = fp if fp else 0
    tn = tn if tn else 0
    
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f1 = 2*(precision*recall/(precision+recall))
    
    return precision*100, recall*100, f1*100


start_c = 0.8
step_c = 0.01
num_c = 20

f_df = pd.DataFrame(columns=['Criteria','Precision(%)','Recall(%)','F1_Score(%)'])

n=0 
while n<num_c:
    criteria = start_c
    
    p,r,f = f_score(criteria)
    
    f_df.loc[len(f_df)] = [criteria, p, r, f]
    
    start_c += step_c
    n+=1
    
f_df

,Criteria,Precision(%),Recall(%),F1_Score(%)
0,0.80,53.571429,80.949811,64.474533
1,0.81,54.125046,78.953049,64.223003
2,0.82,54.645849,77.441986,64.076803
3,0.83,54.901961,75.553157,63.593005
4,0.84,55.228758,72.962763,62.869100
5,0.85,55.659178,70.858068,62.345679
6,0.86,56.121537,67.781975,61.403080
7,0.87,56.975106,65.461414,60.924159
8,0.88,57.568238,62.601187,59.979317
9,0.89,58.206607,59.902860,59.042553


In [118]:
start_c = 0.5
step_c = 0.01
num_c = 20

f_df = pd.DataFrame(columns=['Criteria','Precision(%)','Recall(%)','F1_Score(%)'])

n=0 
while n<num_c:
    criteria = start_c
    
    p,r,f = f_score(criteria)
    
    f_df.loc[len(f_df)] = [criteria, p, r, f]
    
    start_c += step_c
    n+=1
    
f_df

,Criteria,Precision(%),Recall(%),F1_Score(%)
0,0.50,50.675489,99.190502,67.080292
1,0.51,50.690989,98.974636,67.044416
2,0.52,50.705785,98.866703,67.032565
3,0.53,50.805108,98.758770,67.094409
4,0.54,50.890869,98.650836,67.144169
5,0.55,50.933928,98.596870,67.169118
6,0.56,51.048951,98.488937,67.243920
7,0.57,51.096121,98.111171,67.196452
8,0.58,51.169343,98.003238,67.234358
9,0.59,51.229856,97.787372,67.235622


In [ ]:
#Best F1 would be 67.42% but precision only at 51%